## B. Neural Network: MultiClass Classification 
We will modify the architecture that developed in part A to use it for multiclass classification on **Statlog (Vehicle Silhouettes)** Data Set.

More details on the dataset can be found at: https://archive.ics.uci.edu/ml/datasets/Statlog+%28Vehicle+Silhouettes%29

In [1]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from fast_ml.model_development import train_valid_test_split
import sklearn
import sklearn.linear_model
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder


%matplotlib inline

np.random.seed(777)

In [3]:
df = pd.read_csv("Vehicles.csv", header=None)
df.rename(columns = {0:"COMPACTNESS", 1:"CIRCULARITY", 2:"DISTANCE CIRCULARITY", 3:"RADIUS RATIO", 18:"Statlog"},inplace = True)
df.head()

,COMPACTNESS,CIRCULARITY,DISTANCE CIRCULARITY,RADIUS RATIO,4,5,6,7,8,9,10,11,12,13,14,15,16,17,Statlog
0,100,36,73,199,73,6,162,40,20,127,189,401,125,72,6,19,200,204,bus
1,91,36,72,162,60,8,150,44,19,133,166,334,121,63,2,22,196,205,saab
2,91,41,64,148,61,8,129,51,18,142,161,249,153,68,6,12,194,201,van
3,86,39,58,125,55,5,117,57,17,134,140,204,148,69,7,6,190,194,van
4,95,53,95,202,65,10,193,34,22,160,220,559,237,71,3,2,188,196,saab


In [4]:
df.Statlog.value_counts()

bus     218
saab    217
opel    212
van     199
Name: Statlog, dtype: int64

In [5]:
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(df[['Statlog']]).toarray())

Y = enc_df.to_numpy()
Y, Y.shape

(array([[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 (846, 4))

In [7]:
X = df.drop(['Statlog'], axis='columns')
X = X.to_numpy()
X , X.shape

(array([[100,  36,  73, ...,  19, 200, 204],
        [ 91,  36,  72, ...,  22, 196, 205],
        [ 91,  41,  64, ...,  12, 194, 201],
        ...,
        [ 94,  38,  84, ...,  15, 190, 195],
        [104,  52, 100, ...,  10, 191, 198],
        [ 94,  48,  87, ...,  15, 184, 195]], dtype=int64),
 (846, 18))

In [8]:
train_data, X_rem, train_labels, y_rem = train_test_split(X,Y, train_size=0.7)

In [9]:
valid_data, test_data, valid_labels, test_labels = train_test_split(X_rem,y_rem, test_size=0.5)

In [10]:
valid_data

array([[ 86,  44,  77, ...,   0, 187, 192],
       [ 93,  47,  85, ...,   9, 184, 196],
       [ 91,  38,  70, ...,  11, 192, 202],
       ...,
       [ 86,  45,  70, ...,   5, 180, 183],
       [ 82,  38,  53, ...,   0, 177, 183],
       [ 92,  46,  79, ...,  10, 191, 198]], dtype=int64)

In [11]:
scaler = preprocessing.StandardScaler().fit(train_data)
train_data = scaler.transform(train_data)
valid_data = scaler.transform(valid_data)
test_data = scaler.transform(test_data)

In [12]:
trainx = train_data.T
trainy = train_labels.T
validx = valid_data.T
validy = valid_labels.T
testx = test_data.T
testy = test_labels.T

In [13]:
trainx.shape, trainy.shape, validx.shape, validy.shape, testx.shape, testy.shape

((18, 592), (4, 592), (18, 127), (4, 127), (18, 127), (4, 127))

In [14]:
X = trainx
Y = trainy

In [15]:
### START CODE HERE ###
shape_X = X.shape
shape_Y = Y.shape
m = shape_X[1]  # training set size
### END CODE HERE ###

print ('No. of training samples: ' + str(m))
print ('Number of features per sample: ' + str(shape_X[0]))

No. of training samples: 592
Number of features per sample: 18


In [16]:
def model_architecture(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    ### START CODE HERE ### 
    n_x = shape_X[0] # size of input layer
    n_h = 10 
    n_y = shape_Y[0] # size of output layer
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

In [17]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2)

    
    ### START CODE HERE ###
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    ### END CODE HERE ###
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [18]:
def sigmoid(x):
    ### Update THE CODE HERE ###
    return 1/(1 + np.exp(-x))
    ### END CODE HERE ###

In [19]:
def softmax(vector):
    e = np.exp(vector)
    return e / e.sum()

In [20]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### 
    W1 = parameters.get('W1')
    b1 = parameters.get('b1')
    W2 = parameters.get('W2')
    b2 = parameters.get('b2')
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    ### START CODE HERE ### 
    Z1 = np.dot(W1, X) + b1 
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)
    ### END CODE HERE ###
    
    assert(A2.shape == (4, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [21]:
def compute_cost(A2, Y):
    """
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
       
    Returns:
    loss -- cross-entropy loss given equation (7)
    
    """
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    ### START CODE HERE ###
    logprobs = Y * np.log(A2) + (1 - Y) * (np.log(1 - A2))
    cost =  (- 1 / m) * np.sum(logprobs)
    ### END CODE HERE ###
    
    cost = float(np.squeeze(cost))

    assert(isinstance(cost, float))
    
    return cost

In [22]:
def backprop(parameters, cache, X, Y):
    """
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data
    Y -- "true" labels
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### 
    W1 = parameters.get('W1')
    W2 = parameters.get('W2')
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### 
    A1 = cache.get('A1')
    A2 = cache.get('A2')
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### 
    dZ2 = A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [23]:
def update(parameters, grads, learning_rate = 0.01):
    """
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    learning_rate -- The learning rate
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### 
    W1 = parameters.get('W1')
    b1 = parameters.get('b1')
    W2 = parameters.get('W2')
    b2 = parameters.get('b2')
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### 
    dW1 = grads.get('dW1')
    db1 = grads.get('db1')
    dW2 = grads.get('dW2')
    db2 = grads.get('db2')
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### 
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [27]:
def NeuralNetwork(X, Y, n_h, num_iterations = 10000, learning_rate = 0.01, print_loss=False):
    """
    Arguments:
    X -- dataset
    Y -- labels 
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    learning_rate -- The learning rate
    print_loss -- if True, print the loss every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to make predictions.
    """
    
    np.random.seed(3)
    n_x = model_architecture(X, Y)[0]
    n_y = model_architecture(X, Y)[2]
    
    # Initialize parameters
    ### START CODE HERE ### 
    parameters = initialize_parameters(n_x, n_h, n_y)
    ### END CODE HERE ###
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
        ### START CODE HERE ### 
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # loss function. Inputs: "A2, Y, parameters". Outputs: "loss".
        loss = compute_cost(A2, Y)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backprop(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters =  update(parameters, grads, learning_rate = 0.01)
        
        ### END CODE HERE ###
        
        # Print the loss every 100 iterations
        if print_loss and i % 100 == 0:
            print ("loss after iteration %i: %f" %(i, loss))

    return parameters

In [28]:
def predict(parameters, X):
    """
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data 
    
    Returns
    predictions -- vector of predictions of our model
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### 
    A2, cache = forward_propagation(X, parameters)
    
    predictions = np.argmax(A2, axis = 0)
    ### END CODE HERE ###
    
    return predictions

In [53]:
# Build a model with a n_h-dimensional hidden layer
parameters = NeuralNetwork(X, Y, 20, num_iterations = 10000, learning_rate = 0.1, print_loss=False)

In [54]:
# Print accuracy
predictions = predict(parameters, X)
print(accuracy_score(np.argmax(Y.T, axis = 1), predictions.T))

0.6722972972972973


In [55]:
# Print accuracy
predictions_validation = predict(parameters, validx)
print(accuracy_score(np.argmax(validy.T, axis = 1), predictions_validation.T))

0.6220472440944882


In [56]:
# Print accuracy
predictions_test = predict(parameters, testx)
print(accuracy_score(np.argmax(testy.T, axis = 1), predictions_test.T))

0.6535433070866141
